# Normal Estimation
There are cases where normal information is used in point cloud processing.
1. to generate handcrafted features such as PFH, the normal information of each point is required.
2. we compute normals to create 3D object surface information from a scanned point cloud.

If points have only coordinates, we need to estimate normals from points and other information. In this section, we introduce methods to estimate normals from coordinates of points.

This section introduce the following normal estimation methods. 
- Estimation with covariance matrix
- Normal re-orientation method

In [1]:
%load_ext autoreload
%autoreload 2

## Estimation with PCA
In this subsection, we introduce a simple method that estimates normals for each point. This method calculates the third principal component via PCA (Principal Component Analysis) from neighbors of a point.

The code for normal estimation is as follows.

In [2]:
# for normal estimation
import numpy as np
from tutlibs.normal_estimation import normal_estimation, normal_orientation

# for description
from tutlibs.io import Points as io
from tutlibs.transformation import Transformation as tr
from tutlibs.visualization import JupyterVisualizer as jv
import inspect


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Load a point cloud
xyz, _, data = io.read("../data/bunny_pc.ply")

# Estimate normals
k = 15
estimation_normals = normal_estimation(xyz, k=k)

# Get GT normals
gt_normals = np.stack([data["nx"], data["ny"], data["nz"]], axis=-1)

# Visualizaiton
obj_gt_points = jv.point(
    tr.translation(xyz, np.array([1, 0, 0])),  # translation for comparing
    gt_normals,
    color_range=[-1, 1],
    point_size=0.02,
)
obj_estimation_points = jv.point(
    xyz, estimation_normals, color_range=[-1, 1], point_size=0.02
)
jv.display([obj_gt_points, obj_estimation_points])


Output()

In the above output, the point cloud between 0~1 in the x-axis is a normal estimation result, and the point cloud between 1~2 has GT normals. Points are colored by the normal map.
The normal map of estimation result is speckled because the calculation cannot estimate the sign of the normal. In later subsection (Normal re-orientation method), we will introduce how to align the signs of the normals.

Next, we look at the contents of the `normal_estimation` function.

In [4]:
print(inspect.getsource(normal_estimation))


def normal_estimation(coords: np.ndarray, k: int = 10) -> np.ndarray:
    """Estimate normals each point with eigenvector of covariance matrix.
    This function use k nearest neighbor (kNN) to get covariance matrixs.

    Args:
        coords: coordinates of points, (N, 3)
        k: number of neighbor for kNN

    Returns
        normals: estimated normals (N, 3)
    """
    # Get neighbor points. (TODO: add radius)
    idxs, _ = k_nearest_neighbors(coords, coords, k)
    knn_points = gather(coords, idxs)

    # Get covariance matrix of each point.
    knn_mean_points = np.mean(knn_points, axis=-2, keepdims=True)
    deviation = knn_points - knn_mean_points
    deviation_t = deviation.transpose(0, 2, 1)
    covariance_matrixs = np.matmul(deviation_t, deviation) / k  # (N, 3, 3)

    # Get eigenvector and eigenvalue of each point
    w, v = np.linalg.eig(covariance_matrixs.transpose(0, 2, 1))
    # w, v = np.linalg.eig(covariance_matrixs)
    w_min_idxs = np.argmin(w, axis=1)

    # G

In the above implementation, `normal_estimation` returns normals by point. The definition of normal estimation in the above implementation is as follows: given a point cloud $P=[p_1, p_2, ..., p_m, ..., p_M]$ have $M$ points with XYZ coordinates, we estimate normals $N=[n_1, n_2, ..., n_M]$ of points.

The estimation process is as follows:

1. `Get neighbor points.`: 法線を最近傍手法(本subsectionではkNN)を用いて、点群中の各点$p_m$は近傍点$p'_m=[p'_{m_1}, p'_{m_2}, ..., p'_{m_K}]$($p_m$を含む)を取得する。点群中の全ての点の近傍点は$P'=[p'_1, p'_2, ..., p'_m, ..., p'_M]$となる。
2. `Get covariance matrix of each point.`: $p_m$ごとに$p'_m$の点の分布を取得するため、Covariance Matrixを求める。$\overline{p}$を$p'_m$の平均座標値としたとき、Convariance Matrix $c_m$の式は次の通り:
    $$
    \mathcal{C_m}=\frac{1}{K} \sum_{i=1}^{K} \left(\boldsymbol{p}'_{m_i}-\overline{\boldsymbol{p}}\right) \cdot\left(\boldsymbol{p}'_{m_i}-\overline{\boldsymbol{p}}\right)^{T}
    $$
    $c$はM個あるため、$c_1, c_2, ..., c_m, ..., c_M$を取得する。
3. `Get eigenvector and eigenvalue of each point`: 点が最も広がっていない方向を算出するため、$c_m$から固有値と固有ベクトルを取得する。最も低い固有値は点が最も広がっていない方向を指す固有ベクトルに対応する。
4. `Get normal of each point`: 最も低い固有値に対応する固有ベクトルを法線として、法線を割り振る。


## Normal re-orientation method
Estimated normals may not be aligned because the above simple method cannot estimate the sign of the normal. In this subsection, we introduce normal re-orientation method to align direction of normal.

### Minimum spanning tree method
One way to deal with this problem is to use the tree structure of the minimum-wide tree for orientation. We align the normals of points in order, depending on the tree structure.

The code for the method is as follows.

In [4]:
# for normal estimation
import numpy as np
from tutlibs.normal_estimation import normal_estimation, normal_orientation

# for description
from tutlibs.io import Points as io

# from tutlibs.constructing import depth_to_point
from tutlibs.transformation import Transformation as tr
from tutlibs.visualization import JupyterVisualizer as jv
import inspect


In [5]:
# Load a point cloud
xyz, _, data = io.read("../data/bunny_pc.ply")

# Estimate normals
k = 15
estimation_normals = normal_estimation(xyz, k=k)

# with Orientation
ori_estimation_normals = normal_orientation(xyz, estimation_normals)

# Get GT normals
gt_normals = np.stack([data["nx"], data["ny"], data["nz"]], axis=-1)

# visualization
obj_gt_points = jv.point(
    tr.translation(xyz, np.array([2, 0, 0])),  # translation for comparing
    gt_normals,
    color_range=[-1, 1],
    point_size=0.02,
)
obj_estimation_points = jv.point(
    xyz, estimation_normals, color_range=[-1, 1], point_size=0.02
)
obj_ori_estimation_points = jv.point(
    tr.translation(xyz, np.array([1, 0, 0])),  # translation for comparing
    ori_estimation_normals,
    color_range=[-1, 1],
    point_size=0.02,
)
jv.display([obj_gt_points, obj_estimation_points, obj_ori_estimation_points])


Output()

In the above output, the point cloud between 0~1 in the x-axis is a normal estimation result, 1~2 is the re-orienteand points, and 2~3 has GT normals. The re-orientation method is done by the `normal_orientation` function.


### Re-orientation with Viewpoint direction
When acquiring a point cloud from a single viewpoint, it is possible to align the direction of the normal by referring to the viewpoint direction.

Given the viewpoint position $\mathrm{v}_{p}$ and the normal $\overrightarrow{\boldsymbol{n}}_{i}$ of a point $\boldsymbol{p}_{i}$, the new normal $\overrightarrow{\boldsymbol{n'}}_{i}$ is as follows:
<!-- 視点位置vpと点pの法線nがあるとき、新しい法線niは以下の通り。 -->

$$
f(x) = \left\{
\begin{array}{ll}
1 & (x \leqq 0)\\
-1 & (x > 0)
\end{array}
\right.
$$

$$
\overrightarrow{\boldsymbol{n'}}_{i} = \overrightarrow{\boldsymbol{n}}_{i} f(\overrightarrow{\boldsymbol{n}}_{i} \cdot\left(\mathrm{v}_{p}-\boldsymbol{p}_{i}\right))
$$

In [2]:
# for normal estimation
import numpy as np
from tutlibs.normal_estimation import (
    normal_estimation,
    normal_orientation_with_viewpoint,
)

# for description
from tutlibs.io import Points as io
from tutlibs.transformation import Transformation as tr
from tutlibs.visualization import JupyterVisualizer as jv
import inspect


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
# Load a point cloud
coords, _, data = io.read("../data/bunny_pc.ply")
coords = coords - (np.max(coords, axis=0) - np.min(coords, axis=0)) / 2
coords = tr.translation(coords, np.array([0, 0, 1]))

from tutlibs.visualization import JupyterVisualizerUtils as jvu

# Estimate normals
k = 15
estimation_normals = normal_estimation(coords, k=k)

# with Orientation
ori_estimation_normals = normal_orientation_with_viewpoint(
    coords, estimation_normals, np.array([0, 0, 0])
)

# Get GT normals
gt_normals = np.stack([data["nx"], data["ny"], data["nz"]], axis=-1)

# visualization
obj_gt_points = jv.point(
    tr.translation(coords, np.array([1, 0, 0])),  # translation for comparing
    gt_normals,
    color_range=[-1, 1],
    point_size=0.02,
)
obj_estimation_points = jv.point(
    tr.translation(coords, np.array([-1, 0, 0])),  # translation for comparing
    estimation_normals,
    color_range=[-1, 1],
    point_size=0.02,
)
obj_ori_estimation_points = jv.point(
    coords,
    ori_estimation_normals,
    color_range=[-1, 1],
    point_size=0.02,
)
jv.display([obj_gt_points, obj_estimation_points, obj_ori_estimation_points])


Output()

In the above output, a left point cloud in the x-axis is a normal estimation result, a center point cloud is the re-orienteand points, and a right point cloud has GT normals. The re-orientation method is done by the `normal_orientation_with_viewpoint` function. A viewpoint is (0, 0, 0) coordinates.
Note that normal estimation for other side of the object from the viewpoint is not correct.


## References